In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pdf-data/Meetbrain2Music.pdf
/kaggle/input/data4summarization/Meetbrain2Music.pdf
/kaggle/input/data4summarization/Hallucination_in_Large_Language_Models.txt
/kaggle/input/data4summarization/Meetbrain2Mucsic.txt
/kaggle/input/data4summarization/artificial_intelligence_wikipedia.txt


In [2]:
!pip install langchain huggingface_hub tiktoken
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 855.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.7 MB/s eta 0:00:0000:0100:01m


In [26]:
# import os

os.environ["TOGETHER_API_KEY"] = "483eb57ad1aa199b962be798d9116cba588373f8efb8acf918e6b26eeeba42ac"

In [5]:
text_to_summarize = """
New York City comprises 5 boroughs sitting where the Hudson River meets the Atlantic Ocean. At its core is Manhattan, a densely populated borough that’s among the world’s major commercial, financial and cultural centers. Its iconic sites include skyscrapers such as the Empire State Building and sprawling Central Park. Broadway theater is staged in neon-lit Times Square.
"""


In [6]:
with open('/kaggle/input/data4summarization/Meetbrain2Mucsic.txt', 'r', encoding='utf-8') as f:
    conference = f.read()

In [7]:
import fitz 
pdf_path=r"/kaggle/input/pdf-data/Meetbrain2Music.pdf"
doc = fitz.open(pdf_path)
print(len(doc))

10


In [8]:
def extract_section_from_pages(doc, page_start, page_end):
    content = ""
    for page_num in range(page_start, page_end):
        page = doc.load_page(page_num)
        page_text = page.get_text("blocks")
        for block in page_text:
            x0, y0, x1, y1, text = block[:5]
            if not (x1 < 100 and y0 < 100) and not (y0 > page.rect.height - 50):
                content += text + " "
    return content;

In [38]:
reference = extract_section_from_pages(doc, 0, len(doc) - 1);
print(len(reference))

33601


In [39]:
import os
from together import Together

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))
# meta-llama/Llama-3-70b-chat-hf
# meta-llama/Llama-3-8b-chat-hf
response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
      {"role": "system", "content": "You are a helpful assistant that summarizes text. Please provide a detailed summary that is approximately one-half the length of the original text"},
      {"role": "user", "content": f"Please summarize the following text: {reference}"},
    ],
)


print(response.choices[0].message.content)
candidate = response.choices[0].message.content

Here is a detailed summary of the text:

**Introduction**

The paper explores the possibility of reconstructing music from brain activity using a music generation model called MusicLM. The authors investigate whether it is possible to predict music embeddings from fMRI scans and use them to generate music that semantically resembles the original music stimulus.

**Methods**

The authors use a music dataset with 540 music pieces from 10 genres, each with a 15-second clip. They also use a text caption dataset with descriptions of the music pieces. The MusicLM model is used to generate music conditioned on a MuLan embedding, which is a joint text/music embedding model. The authors predict the MuLan embedding from fMRI scans using a linear regression model and then use the predicted embedding to generate music.

**Decoding Results**

The authors report the identification accuracy of the predicted music embeddings and find that MuLanmusic embeddings can be more accurately predicted from fMR

In [11]:
!pip install nltk

In [12]:
from nltk.translate.bleu_score import sentence_bleu

In [28]:
score = sentence_bleu(conference.split(), candidate.split())
print(score)

0.27673806768630216


In [14]:
!pip install tensorflow tensorflow-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [15]:
import tensorflow as tf
import tensorflow_hub as hub

# Load Universal Sentence Encoder model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

embeddings = model([conference, candidate])

similarity_score = tf.tensordot(embeddings[0], embeddings[1], axes=1).numpy()

print(f"Similarity score between summary_MarkTechPost and summary: {similarity_score}")


2024-07-18 15:46:32.500974: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 15:46:32.501084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 15:46:32.683877: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Similarity score between summary_MarkTechPost and summary: 0.7701590061187744


In [16]:
def calculateSemanticSimilarity(conference, candidate):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)

    embeddings = model([conference, candidate])

    similarity_score = tf.tensordot(embeddings[0], embeddings[1], axes=1).numpy()
    return similarity_score

In [29]:
print (calculateSemanticSimilarity(conference, candidate))

0.83325404


In [18]:
!pip install openai tiktoken tqdm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.4 MB/s eta 0:00:00a 0:00:01m


In [19]:
import os
from typing import List, Tuple, Optional
from openai import OpenAI
import tiktoken
from tqdm import tqdm

In [20]:
def get_chat_completion(messages, model='meta-llama/Llama-3-8b-chat-hf'):
    response1 = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response1.choices[0].message.content

In [21]:
def tokenize(text: str) -> List[str]:
    encoding = tiktoken.encoding_for_model('gpt-4-turbo')
    return encoding.encode(text)


# This function chunks a text into smaller pieces based on a maximum token count and a delimiter.
def chunk_on_delimiter(input_string: str,
                       max_tokens: int, delimiter: str) -> List[str]:
    chunks = input_string.split(delimiter)
    combined_chunks, _, dropped_chunk_count = combine_chunks_with_no_minimum(
        chunks, max_tokens, chunk_delimiter=delimiter, add_ellipsis_for_overflow=True
    )
    if dropped_chunk_count > 0:
        print(f"warning: {dropped_chunk_count} chunks were dropped due to overflow")
    combined_chunks = [f"{chunk}{delimiter}" for chunk in combined_chunks]
    return combined_chunks


# This function combines text chunks into larger blocks without exceeding a specified token count. It returns the combined text blocks, their original indices, and the count of chunks dropped due to overflow.
def combine_chunks_with_no_minimum(
        chunks: List[str],
        max_tokens: int,
        chunk_delimiter="\n\n",
        header: Optional[str] = None,
        add_ellipsis_for_overflow=False,
) -> Tuple[List[str], List[int]]:
    dropped_chunk_count = 0
    output = []  # list to hold the final combined chunks
    output_indices = []  # list to hold the indices of the final combined chunks
    candidate = (
        [] if header is None else [header]
    )  # list to hold the current combined chunk candidate
    candidate_indices = []
    for chunk_i, chunk in enumerate(chunks):
        chunk_with_header = [chunk] if header is None else [header, chunk]
        if len(tokenize(chunk_delimiter.join(chunk_with_header))) > max_tokens:
            print(f"warning: chunk overflow")
            if (
                    add_ellipsis_for_overflow
                    and len(tokenize(chunk_delimiter.join(candidate + ["..."]))) <= max_tokens
            ):
                candidate.append("...")
                dropped_chunk_count += 1
            continue  # this case would break downstream assumptions
        # estimate token count with the current chunk added
        extended_candidate_token_count = len(tokenize(chunk_delimiter.join(candidate + [chunk])))
        # If the token count exceeds max_tokens, add the current candidate to output and start a new candidate
        if extended_candidate_token_count > max_tokens:
            output.append(chunk_delimiter.join(candidate))
            output_indices.append(candidate_indices)
            candidate = chunk_with_header  # re-initialize candidate
            candidate_indices = [chunk_i]
        # otherwise keep extending the candidate
        else:
            candidate.append(chunk)
            candidate_indices.append(chunk_i)
    # add the remaining candidate to output if it's not empty
    if (header is not None and len(candidate) > 1) or (header is None and len(candidate) > 0):
        output.append(chunk_delimiter.join(candidate))
        output_indices.append(candidate_indices)
    return output, output_indices, dropped_chunk_count

In [22]:
def summarize(text: str,
              detail: float = 0,
              model: str = 'gpt-4-turbo',
              additional_instructions: Optional[str] = None,
              minimum_chunk_size: Optional[int] = 500,
              chunk_delimiter: str = ".",
              summarize_recursively=False,
              verbose=False):
    """
    Summarizes a given text by splitting it into chunks, each of which is summarized individually. 
    The level of detail in the summary can be adjusted, and the process can optionally be made recursive.

    Parameters:
    - text (str): The text to be summarized.
    - detail (float, optional): A value between 0 and 1 indicating the desired level of detail in the summary.
      0 leads to a higher level summary, and 1 results in a more detailed summary. Defaults to 0.
    - model (str, optional): The model to use for generating summaries. Defaults to 'phamtungthuy/Vistral-7B-Chat-AWQ'.
    - additional_instructions (Optional[str], optional): Additional instructions to provide to the model for customizing summaries.
    - minimum_chunk_size (Optional[int], optional): The minimum size for text chunks. Defaults to 500.
    - chunk_delimiter (str, optional): The delimiter used to split the text into chunks. Defaults to ".".
    - summarize_recursively (bool, optional): If True, summaries are generated recursively, using previous summaries for context.
    - verbose (bool, optional): If True, prints detailed information about the chunking process.

    Returns:
    - str: The final compiled summary of the text.

    The function first determines the number of chunks by interpolating between a minimum and a maximum chunk count based on the `detail` parameter. 
    It then splits the text into chunks and summarizes each chunk. If `summarize_recursively` is True, each summary is based on the previous summaries, 
    adding more context to the summarization process. The function returns a compiled summary of all chunks.
    """

    # check detail is set correctly
    assert 0 <= detail <= 1

    # interpolate the number of chunks based to get specified level of detail
    max_chunks = len(chunk_on_delimiter(text, minimum_chunk_size, chunk_delimiter))
    min_chunks = 1
    num_chunks = int(min_chunks + detail * (max_chunks - min_chunks))

    # adjust chunk_size based on interpolated number of chunks
    document_length = len(tokenize(text))
    chunk_size = max(minimum_chunk_size, document_length // num_chunks)
    text_chunks = chunk_on_delimiter(text, chunk_size, chunk_delimiter)
    if verbose:
        print(f"Splitting the text into {len(text_chunks)} chunks to be summarized.")
        print(f"Chunk lengths are {[len(tokenize(x)) for x in text_chunks]}")

    # set system message
    system_message_content = "Rewrite this text in summarized form."
    if additional_instructions is not None:
        system_message_content += f"\n\n{additional_instructions}"

    accumulated_summaries = []
    for chunk in tqdm(text_chunks):
        if summarize_recursively and accumulated_summaries:
            # Creating a structured prompt for recursive summarization
            accumulated_summaries_string = '\n\n'.join(accumulated_summaries)
            user_message_content = f"Previous summaries:\n\n{accumulated_summaries_string}\n\nText to summarize next:\n\n{chunk}"
        else:
            # Directly passing the chunk for summarization without recursive context
            user_message_content = chunk

        # Constructing messages based on whether recursive summarization is applied
        messages = [
            {"role": "system", "content": system_message_content},
            {"role": "user", "content": user_message_content}
        ]

        # Assuming this function gets the completion and works as expected
        response = get_chat_completion(messages)
        accumulated_summaries.append(response)

    # Compile final summary from partial summaries
    final_summary = '\n\n'.join(accumulated_summaries)

    return final_summary

In [30]:
summary_with_detail_1 = summarize(reference, detail=1, verbose=True)

Splitting the text into 16 chunks to be summarized.
Chunk lengths are [485, 501, 491, 418, 482, 495, 486, 492, 493, 480, 498, 499, 480, 501, 490, 404]


100%|██████████| 16/16 [00:19<00:00,  1.23s/it]


In [31]:
score1 = sentence_bleu(conference.split(), summary_with_detail_1.split())
print(score1)

0.19448257948621436


In [32]:
calculateSemanticSimilarity(conference,summary_with_detail_1)

0.77080715